<a href="https://colab.research.google.com/github/sevann-radhak/uba-aprendizaje-profundo/blob/main/NLP_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [ ]:
import os
import pandas as pd
from zipfile import ZipFile
import matplotlib.pyplot as plt
import seaborn as sns
import platform

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [ ]:
# Verificar si el archivo ya ha sido descargado
if os.access('./fake_news_dataset/Fake.csv', os.F_OK) is False:
    if os.access('Fake.csv', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/sevann-radhak/procesamiento_lenguaje_natural/main/clase_2/ejercicios/raw/Fake.csv -o Fake.csv
        else:
            !wget https://raw.githubusercontent.com/sevann-radhak/procesamiento_lenguaje_natural/main/clase_2/ejercicios/raw/Fake.csv -O Fake.csv
    # Crear la carpeta y mover el archivo
    os.makedirs('./fake_news_dataset', exist_ok=True)
    os.rename('Fake.csv', './fake_news_dataset/Fake.csv')
else:
    print("El archivo ya se encuentra descargado")

--2024-07-04 23:20:28--  https://raw.githubusercontent.com/sevann-radhak/procesamiento_lenguaje_natural/main/clase_2/ejercicios/raw/Fake.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62789876 (60M) [text/plain]
Saving to: ‘Fake.csv’

Fake.csv            100%[===================>]  59.88M  92.0MB/s    in 0.7s    

2024-07-04 23:20:32 (92.0 MB/s) - ‘Fake.csv’ saved [62789876/62789876]



In [ ]:
# # Posibles bandas
# os.listdir("./songs_dataset/")

In [ ]:
df = pd.read_csv('./fake_news_dataset/Fake.csv')
print(df.head(5))

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  


In [ ]:
# Combinar las columnas title y text en una sola columna llamada content
df['content'] = df.apply(lambda row: row['title'] + '.' + row['text'] if not row['title'].endswith('.') else row['title'] + ' ' + row['text'], axis=1)

# Eliminar las columnas date y subject
df.drop(columns=['date', 'subject', 'title', 'text'], inplace=True)

# Mostrar el resultado del DataFrame
print(df.head(5))

                                             content
0   Donald Trump Sends Out Embarrassing New Year’...
1   Drunk Bragging Trump Staffer Started Russian ...
2   Sheriff David Clarke Becomes An Internet Joke...
3   Trump Is So Obsessed He Even Has Obama’s Name...
4   Pope Francis Just Called Out Donald Trump Dur...


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 23481


### 1 - Preprocesamiento

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:5]

[['donald',
  'trump',
  'sends',
  'out',
  'embarrassing',
  'new',
  'year’s',
  'eve',
  'message',
  'this',
  'is',
  'disturbing',
  'donald',
  'trump',
  'just',
  'couldn',
  't',
  'wish',
  'all',
  'americans',
  'a',
  'happy',
  'new',
  'year',
  'and',
  'leave',
  'it',
  'at',
  'that',
  'instead',
  'he',
  'had',
  'to',
  'give',
  'a',
  'shout',
  'out',
  'to',
  'his',
  'enemies',
  'haters',
  'and',
  'the',
  'very',
  'dishonest',
  'fake',
  'news',
  'media',
  'the',
  'former',
  'reality',
  'show',
  'star',
  'had',
  'just',
  'one',
  'job',
  'to',
  'do',
  'and',
  'he',
  'couldn',
  't',
  'do',
  'it',
  'as',
  'our',
  'country',
  'rapidly',
  'grows',
  'stronger',
  'and',
  'smarter',
  'i',
  'want',
  'to',
  'wish',
  'all',
  'of',
  'my',
  'friends',
  'supporters',
  'enemies',
  'haters',
  'and',
  'even',
  'the',
  'very',
  'dishonest',
  'fake',
  'news',
  'media',
  'a',
  'happy',
  'and',
  'healthy',
  'new',
  'yea

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.loss_previous_step = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print(f'Loss after epoch {self.epoch}: {loss - self.loss_previous_step}')
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# w2v_model = Word2Vec(min_count=2,
#                      window=5,
#                      vector_size=300,
#                      negative=10,
#                      workers=multiprocessing.cpu_count(),
#                      sg=1)

w2v_model = Word2Vec(min_count=5,
                     window=3,
                     vector_size=300,
                     negative=20,
                     workers=multiprocessing.cpu_count(),
                     sg=1,
                     alpha=0.01,
                     min_alpha=0.0001)

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 23481


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 36929


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=10,
                 compute_loss=True,
                 callbacks=[callback()])

Loss after epoch 0: 39330284.0
Loss after epoch 1: 30409308.0
Loss after epoch 2: 9325760.0
Loss after epoch 3: 9151904.0
Loss after epoch 4: 8984208.0
Loss after epoch 5: 8797904.0
Loss after epoch 6: 8669640.0
Loss after epoch 7: 8486584.0
Loss after epoch 8: 8298552.0
Loss after epoch 9: 2763584.0


(83167146, 105970810)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('neanderthal', 0.5222920179367065),
 ('beck’s', 0.5124498605728149),
 ('filibustering', 0.5115554928779602),
 ('lamestream', 0.5015192031860352),
 ('talker', 0.4978712797164917),
 ('inarguably', 0.4959251284599304),
 ('partier', 0.49426954984664917),
 ('braindead', 0.486009806394577),
 ('fanboy', 0.4856387972831726),
 ('office”', 0.48508042097091675)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('pipeline', 0.029985863715410233),
 ('heading', 0.022912006825208664),
 ('bay', 0.0223403237760067),
 ('resulting', 0.017341097816824913),
 ('within', 0.016021868214011192),
 ('listed', 0.014005560427904129),
 ('fines', 0.011431209743022919),
 ('permanently', 0.010134845972061157),
 ('formerly', 0.009056088514626026),
 ('detention', 0.00858086347579956)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["donald"], topn=10)

[('trump', 0.6394109725952148),
 ('election’', 0.5323485136032104),
 ('“president', 0.5260996222496033),
 ('hesitance', 0.5223474502563477),
 ('tyra', 0.520029604434967),
 ('2016donald', 0.5179675221443176),
 ('substantively', 0.5168504118919373),
 ('wyunhjjujg', 0.516535758972168),
 ('flirted', 0.5135263204574585),
 ('pussygrabber', 0.511838972568512)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["russia"], topn=10)

[('russian', 0.6540964841842651),
 ('russians', 0.6155077219009399),
 ('putin', 0.568894624710083),
 ('kremlin', 0.5344482660293579),
 ('hacking', 0.5285258293151855),
 ('meddling', 0.5062726140022278),
 ('vladamir', 0.5005854964256287),
 ('collusion', 0.4980163872241974),
 ('crimean', 0.4947540760040283),
 ('gru', 0.4921318590641022)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["africa"], topn=5)

[('bahamas', 0.4764396548271179),
 ('saharan', 0.45500367879867554),
 ('yugoslavia', 0.451762855052948),
 ('botswana', 0.44585078954696655),
 ('drc', 0.44478172063827515)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
# w2v_model.wv.most_similar(negative=["diedaa"])

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector = w2v_model.wv.get_vector("africa")
print(vector)

[ 0.19864179  0.1622068  -0.19305669  0.11801786 -0.13735205 -0.29887536
 -0.6846458   0.3276352   0.3858823  -0.2060942   0.02855772  0.02571031
 -0.48876026  0.2926561  -0.49688515  0.04458421  0.57708794  0.53378564
 -0.07771533 -0.1557589   0.6732586   0.0257975   0.30954698  0.06503145
  0.11390983 -0.4404625  -0.26089063  0.11032988 -0.14421283 -0.21778238
  0.23507844 -0.37715557 -0.26627117  0.13511638 -0.51345897 -0.103202
 -0.10180385 -0.31726834  0.4641404  -0.24500878 -0.23398292 -0.09535668
 -0.64072007 -0.4162411  -0.06713346  0.53932434  0.13830753 -0.01976629
 -0.17610368  0.27606472  0.0522093   0.32918993  0.20070213  0.652891
  0.65916044 -0.05833929  0.62520105 -0.03257269  0.32781437  0.21376084
 -0.46414307 -0.2844595   0.08334647  0.08107897 -0.2695733  -0.14962073
  0.80022883 -0.17304301 -0.28195718 -0.03097164 -0.04979676  0.26161346
 -0.05762055 -0.0019173  -0.40336198  0.19535327 -0.48240054  0.0079421
  0.0898842   0.528935    0.05603101 -0.4226354  -0.0480

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector)

[('africa', 1.0),
 ('bahamas', 0.4764396548271179),
 ('saharan', 0.4550037384033203),
 ('yugoslavia', 0.451762855052948),
 ('botswana', 0.4458507001399994),
 ('drc', 0.44478172063827515),
 ('europe', 0.44114476442337036),
 ('deserts', 0.4402143657207489),
 ('lesotho', 0.43877723813056946),
 ('burundi', 0.4357914924621582)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# # Graficar los embedddings en 3D

# vecs, labels = reduce_dimensions(w2v_model,3)

# fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
# fig.update_traces(marker_size = 2)
# fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.